# SST in Hurricane Irene
  
Authors
* [Dr Chelle Gentemann](mailto:gentemann@esr.org)    - Earth and Space Research, USA
* [Dr Marisol Garcia-Reyes](mailto:marisolgr@faralloninstitute.org)  - Farallon Institute, USA 
* PODAACPY file search added by [Lewis John McGibbney](mailto:lewis.j.mcgibbney@jpl.nasa.gov)  -JPL, NASA, USA
  

-------------------

## Import python packages

* You are going to want numpy, pandas, matplotlib.pyplot, podaaacpy, and xarray
* This cell also imports a parser so that a login file can be read to use podaacpy

In [ ]:
import warnings
warnings.simplefilter('ignore') # filter some warning messages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs

#This is for reading in and parsing the login file credentials
from pathlib import Path
import configparser
from lxml import objectify

#The podaacpy api
import podaac.podaac as podaac
from podaac import drive as podaacdrive
import podaac.podaac_utils as putil
# then create an instance of the Podaac class
p = podaac.Podaac()

#read in the login credentials and pass them to podaac drive
with open('./../podaac.ini', 'r') as f:
    config = configparser.ConfigParser()
    config.read_file(f)
    d = podaacdrive.Drive(None, 
                          config['drive']['urs_username'], 
                          config['drive']['urs_password'])

## Analysis of SSTs during Hurricane Irene

Irene was a massive storm, with tropical storm force winds extending outward 300 miles (485 km). The storm was also slow moving as it traversed the Mid-Atlantic. Irene claimed at least 48 lives and caused over 7 billion U.S. dollars in damages in the U.S. and 3.1 billion U.S. dollars of damage in the Caribbean. (source: https://www.ncdc.noaa.gov/sotc/tropical-cyclones/201113).

For this tutorial we will use the podaacpy to search for SST2 during Hurricane Irene and look at the change in upper ocean heat content and chlorophyll-a.  https://www.livescience.com/30759-how-a-hurricane-impacts-the-ocean.html

For plotting the data, try using cartopy
`ax = plt.axes(projection=ccrs.Orthographic(-70, 30))`
you will need to add  `transform=ccrs.PlateCarree()` to you plotting routine
`ax.set_extent([-82, -50, 10, 60], crs=ccrs.PlateCarree())`
`ax.coastlines('50m')`
`ax.stock_img()`


In [ ]:
url = 'http://mrtee.europa.renci.org:8080/thredds/dodsC/DataLayers/IBTrACS.NA.v04r00.nc?name[0:1:2211],time[0:1:2211][0:1:359],lat[0:1:2211][0:1:359],lon[0:1:2211][0:1:359]'

ds_storm=xr.open_dataset(url)

irene = ds_storm.isel(storm=2092).isel(date_time=slice(0,78))

plt.scatter(irene.lon,irene.lat,c=irene.time.dt.dayofyear)

print('storm start and end:', irene.time[0].data,irene.time[-1].data)

In [ ]:
# try plotting here with land mask

In [ ]:
start_time = '2011-08-20T00:00:00Z'
end_time = '2011-09-05T23:59:59Z'
start_time2 = '2011-08-15'
end_time2 = '2011-09-25'

minlat,maxlat = 15,45
minlon,maxlon = -100,-40

#dataset_id = 'PODAAC-GHCMC-4FM03'  #CMC SST looked up on podaac website, on dataset page this is the persistant id
#dataset_id = 'PODAAC-GHCMC-4FM02'  #CMC SST looked up on podaac website, on dataset page this is the persistant id
#dataset_id = 'PODAAC-GHGMR-4FJ04'  #MUR SST
#dataset_id = 'PODAAC-GHGDM-4FD02'  #DMI SST
#dataset_id = 'PODAAC-GHGPB-4FO02'  #ospo sst

dataset_id = 'PODAAC-GHCMC-4FM02'  #CMC SST
gresult = p.granule_search(dataset_id=dataset_id,
                           start_time=start_time,
                           end_time=end_time,
                           items_per_page='100')
urls = putil.PodaacUtils.mine_opendap_urls_from_granule_search(gresult)
urls_sst = [w[:-5] for w in urls]  #remove html from urlsurls_sst = [w.replace('-tools.jpl.nasa.gov/drive/files/', '-opendap.jpl.nasa.gov/opendap/') for w in urls_sst]
print('num files:',len(urls_sst))


In [ ]:
ds_sst = xr.open_dataset(urls_sst[0])

subset_sst = ds_sst.sel(lat=slice(minlat,maxlat)
                        ,lon=slice(minlon,maxlon))

print('opening:', urls_sst[0],subset_sst)
#subset_sst.analysed_sst.plot()

In [ ]:
fig, axes = plt.subplots(ncols=2,figsize=[12,4])

subset_sst.analysed_sst[0,:,:].plot(ax=axes[0])
axes[0].scatter(irene.lon[0:78],irene.lat[0:78],c=irene.time.dt.dayofyear[0:78],cmap='seismic')

subset_sst.mask[0,:,:].plot(ax=axes[1])
axes[0].scatter(irene.lon[0:78],irene.lat[0:78],c=irene.time.dt.dayofyear[0:78],cmap='seismic')


## Mask out land values using .where

In [ ]:
subset_sst_masked = subset_sst.where(subset_sst.mask==1)

subset_sst_masked.analysed_sst[0,:,:].plot()


## Compare time series of the cold wake after Hurricane as measured by MUR and OSTIA SSTs

## When you open a multi-file dataset, xarray uses dask for lasy loading.  
* Lazy loading: It mostly just loads the metadata. You can do data searching, selecting, subsetting without acutally loading the data. 
* Here we have loaded in 14 days of data for a very high resolution SST global datasets.  Before we actually load the data, we are going to want to do some subsetting so that it will fit into our memory.
* Notice below when you print out the dataset details that they are all stored as dask.array types.

In [ ]:
ds_sst = xr.open_mfdataset(urls_sst,coords='minimal')

ds_sst = ds_sst.where(ds_sst.mask==1)

#subset data
subset_sst = ds_sst.sel(lat=slice(minlat,maxlat),
                        lon=slice(minlon,maxlon))


# Check the size of the data 

In [ ]:
print('GB of data:', subset_sst.nbytes/1e9)

In [ ]:
#load the data
subset_sst.load()


# Load chlorophyll-a data from a virtually aggregated dataset at COASTWATCH
* subset data

In [ ]:
url = 'https://coastwatch.pfeg.noaa.gov/erddap/griddap/pmlEsaCCI31OceanColorDaily'
ds_chl = xr.open_dataset(url).rename({'latitude':'lat','longitude':'lon'}) 

ds_chl_subset = ds_chl.sel(time=slice(start_time2,end_time2),
                           lat=slice(45,15),
                           lon=slice(-100,-40))
                        

chl = ds_chl_subset.chlor_a.sortby('lat')



* Create a 5-day resampled dataset since the chl-a data is missing when clouds are present

In [ ]:
chl_5dy = chl.resample(time='5D').mean('time')

* Interpolate onto daily maps

In [ ]:
chl_1dy = chl_5dy.resample(time='1D').interpolate('linear')

# Look at the Chlorophyll-a data

In [ ]:
fig, axes = plt.subplots(ncols=2,figsize=[12,4])

chl_5dy[0,:,:].plot(vmin=0,vmax=.5,ax=axes[0])

axes[0].scatter(irene.lon,irene.lat,
                c=irene.time.dt.dayofyear,
                cmap='seismic')

(chl_5dy[4,:,:]-chl_5dy[0,:,:]).plot(vmin=-0.2,vmax=.2,ax=axes[1],cmap='seismic')

axes[1].scatter(irene.lon,irene.lat,
                c=irene.time.dt.dayofyear,
                cmap='jet')

axes[1].grid()

In [ ]:
chl_ts = chl_1dy.sel(lat=32.0,method='nearest').sel(lon=-80.0,method='nearest')
chl_ts.plot()

In [ ]:

#first interpolate onto same time sampling
subset_chl_interp_time = chl_1dy.interp(time=subset_sst.time,
                                      method='linear')

#now interpolate onto same spatial grid
subset_chl_interp = subset_chl_interp_time.interp(lat=subset_sst.lat,
                                      lon=subset_sst.lon,
                                      method='nearest')

#now mask the data
subset_chl_masked = subset_chl_interp.where(subset_sst.mask==1)



In [ ]:
fig, axes = plt.subplots(ncols=2,figsize=[12,4])

dif = (subset_sst.analysed_sst[10,:,:]-subset_sst.analysed_sst[0,:,:])

dif.plot(vmin=-1,vmax=1,ax=axes[0],cmap='seismic')

dif2 = (subset_chl_masked[11,:,:]-subset_chl_masked[0,:,:])

dif2.plot(vmin=-1,vmax=1,ax=axes[1],cmap='seismic')



In [ ]:

f = plt.figure(figsize=(12,4))

dif = (subset_sst.analysed_sst[10,:,:]-subset_sst.analysed_sst[0,:,:])

ax1 = plt.subplot(121, projection=ccrs.Orthographic(-70, 30))

dif.plot(vmin=-1,vmax=1,ax=ax1,cmap='seismic',transform=ccrs.PlateCarree())
ax1.set_extent([-82, -50, 15, 45], crs=ccrs.PlateCarree())
ax1.coastlines('50m')
ax1.stock_img()


dif2 = (subset_chl_masked[11,:,:]-subset_chl_masked[0,:,:])

ax2 = plt.subplot(122, projection=ccrs.Orthographic(-70, 30))

(dif*0).plot(vmin=-1,vmax=1,ax=ax2,cmap='seismic',transform=ccrs.PlateCarree(),add_colorbar=False)
dif2.plot(vmin=-1,vmax=1,ax=ax2,cmap='seismic',transform=ccrs.PlateCarree())
ax2.set_extent([-82, -50, 15, 45], crs=ccrs.PlateCarree())
ax2.coastlines('50m')
ax2.stock_img()

